# Desenvolvimento da Carteira Consolidada

**Preparado por:** Vinicius Suguimoto

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

import aurum
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Teoricamente toda essa etapa de treinamento do modelo já estará encapsulada no
# modelo salvo na pasta models, contudo a partir desse diretório não consigo acessar
# a pasta. Treinaremos um modelo simples para consolidar os resultados

def treinar_modelo(TICKER):
    DATA = aurum.data.get_ohlcv(TICKER)
    DATA = aurum.ft.technical_indicators(DATA)

    TRAIN_SIZE = int(len(DATA) * .75)

    INDICADORES = [
        'RSI_14',
        'STOCHk_14_3_3',
        'STOCHd_14_3_3',
        'ROC_2',
        'ROC_5',
        'ROC_10',
        'SLOPE_3',
        'ATRr_5',
        'WILLR_14',
        'OBV',
        'OBV_ROC_14',
        'EMA_BUY_CROSS',
        'EMA_SELL_CROSS',
        'EMA_9_DISTANCE', 
        'EMA_21_DISTANCE',
        'BBAND_FechouFora_Lower',
        'BBAND_FechouFora_Upper'
    ]

    X_train = DATA.loc[TRAIN_SIZE:, INDICADORES]
    y_train = DATA.loc[TRAIN_SIZE:, 'Alvo']

    MODEL = aurum.model_training.create_setup(X_train, y_train)

    return MODEL

# Modelos para a Carteira

MODELO_PETR4 = treinar_modelo('PETR4.SA')
MODELO_CMIG3 = treinar_modelo('CMIG3.SA')
MODELO_ABEV3 = treinar_modelo('ABEV3.SA')

In [5]:
# Resultados consolidados

def model_results(TICKER, MODEL): 

    TEST = aurum.data.get_ohlcv(TICKER, TREINO=False)
    TEST = aurum.ft.technical_indicators(TEST)
    TEST['Predicao'] = MODEL.predict(TEST[MODEL.feature_names_in_])

    TEST['Ticker'] = TICKER
    TEST['Retorno do Modelo'] = TEST['Predicao'] * TEST['LEAK_Retorno']/4

    RETURN = TEST[['Ticker', 'Date', 'Predicao', 'Retorno do Modelo']]

    return RETURN



In [6]:
RETORNO_PETR4 = model_results('PETR4.SA', MODELO_PETR4)
RETORNO_CMIG3 = model_results('CMIG3.SA', MODELO_CMIG3)
RETORNO_ABEV3 = model_results('ABEV3.SA', MODELO_ABEV3)

In [7]:
def wallet_return(RETURN_LIST, TICKER_WEIGHT):

    assert len(RETURN_LIST) == len(TICKER_WEIGHT)
    assert sum(TICKER_WEIGHT) == 1

    
    RETORNO_PONDERADO_MODELO = np.zeros(len(RETURN_LIST[0]))

    for i, retorno in enumerate(RETURN_LIST):

        RETORNO_PONDERADO_MODELO += retorno['Retorno do Modelo'] * TICKER_WEIGHT[i]

    RETORNO_ACUMULADO_MODELO = (1 + RETORNO_PONDERADO_MODELO).cumprod()

    RESULTADOS = pd.DataFrame(RETORNO_ACUMULADO_MODELO)
    RESULTADOS['Date'] = RETURN_LIST[0]['Date']
    RESULTADOS = RESULTADOS.rename({'Retorno do Modelo': 'Retorno da Carteira'}, axis=1)

    return RESULTADOS[['Date', 'Retorno da Carteira']]

In [8]:
RETORNO_GERAL = wallet_return([RETORNO_PETR4, RETORNO_CMIG3], [.5, .5])

In [9]:
import plotly.express as px

In [10]:
px.line(RETORNO_GERAL,x='Date', y='Retorno da Carteira')

---

In [ ]:
def retorno_consolidado(LISTA_RETORNOS):

In [21]:
lista_retornos = [RETORNO_PETR4, RETORNO_CMIG3, RETORNO_ABEV3]
consolidado = pd.concat(lista_retornos)
consol_pivot = consolidado.pivot_table(index='Date', columns=['Ticker'], values=['Predicao', 'Retorno do Modelo'])

In [45]:
consol_pivot.tail(10)

Predicao                   Retorno do Modelo                    
Ticker     ABEV3.SA CMIG3.SA PETR4.SA          ABEV3.SA  CMIG3.SA  PETR4.SA
Date                                                                       
2022-10-28        1        0        1          0.009353  0.000000 -0.041603
2022-10-31        1        1        1          0.011562  0.014170 -0.020631
2022-11-01        1        1        1          0.010925 -0.003882 -0.024782
2022-11-03        1        1        1         -0.000468 -0.022329 -0.031970
2022-11-04        0        1        1          0.000000 -0.023081 -0.011572
2022-11-07        1        0        1               NaN       NaN       NaN
2022-11-08        0        0        1               NaN       NaN       NaN
2022-11-09        1        0        1               NaN       NaN       NaN
2022-11-10        1        0        1               NaN       NaN       NaN
2022-11-11        1        1        1               NaN       NaN       NaN

In [42]:
consol_pivot.iloc[-2, :].loc[('Predicao', 'CMIG3.SA')]

0.0